In [1]:
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import os
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
from tqdm import tqdm, trange
import time
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, cohen_kappa_score
from efficientnet_pytorch import EfficientNet
import pandas as pd

In [2]:
seed = 42
torch.cuda.get_device_name()

'Tesla T4'

In [3]:
labels = pd.read_csv("../input/training-labels.csv")

In [4]:
labels.groupby(['Drscore']).size()

Drscore
0    11651
1     1397
2     3299
3      841
4      619
dtype: int64

In [5]:
train_df, val_df = train_test_split(labels, test_size=0.2,stratify=labels['Drscore'], random_state = seed)

In [6]:
BATCH_SIZE = 2**3
NUM_WORKERS = 2
LEARNING_RATE = 5e-5
NUM_EPOCHS = 8
LOG_FREQ = 100
TIME_LIMIT = 8 * 60 * 60
RESIZE = 350
torch.cuda.empty_cache()

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# classes

In [8]:
class ORCNN(nn.Module):
    def __init__(self):
        super(ORCNN, self).__init__()
        self.efnet = EfficientNet.from_pretrained('efficientnet-b4', num_classes=5)
        self.efnet._fc = nn.Identity()
        self.or_layers = nn.ModuleList()
        for k in range(4):
            self.or_layers.append(nn.Linear(1792, 2))

    def forward(self, x, k):
        x = self.efnet(x)
        x = self.or_layers[k](x)
        
        return x

In [9]:
class ImageDataset(Dataset):
    def __init__(self, dataframe, mode):
        assert mode in ['train', 'val', 'test']

        self.df = dataframe.copy()
#         self.df['Drscore'] = (self.df['Drscore'] > k).astype(int)
        self.mode = mode

        transforms_list = [
            transforms.Resize(RESIZE),
            transforms.CenterCrop(RESIZE)
        ]

        if self.mode == 'train':
            transforms_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomChoice([
                    transforms.RandomAffine(degrees=(0,360), translate=(0.1, 0.1),
                                            scale=(0.9, 1.1))
                ])
            ])

        transforms_list.extend([
            transforms.ToTensor(),
            #             transforms.Normalize(mean=[0.485, 0.456, 0.406],
            #                                   std=[0.229, 0.224, 0.225]),
        ])
        self.transforms = transforms.Compose(transforms_list)

    def __getitem__(self, index):
        ''' Returns: tuple (sample, target) '''
        filename = self.df['Filename'].values[index]

        directory = '../input/Test' if self.mode == 'test' else '../input/output_combined2'
        sample = Image.open(f'./{directory}/gb_{filename}')
        image = self.transforms(sample)

        if self.mode == 'test':
            return image
        else:
            return image, self.df['Drscore'].values[index]

    def scaleRadius(self, img, scale):
        x = img[img.shape[0]//2,:,:].sum(1)
        r = (x > x.mean() / 10).sum() / 2
        s = scale * 1.0 / r
        return cv2.resize(img, (0, 0), fx=s, fy=s)

    def __len__(self):
        return self.df.shape[0]

In [10]:
def GAP(predicts, confs, targets):
    ''' Simplified GAP@1 metric: only one prediction per sample is supported '''
    predicts = predicts.cpu().numpy()
    targets = targets.cpu().numpy()

    res = accuracy_score(targets, predicts)
    return res

In [11]:
class AverageMeter:
    ''' Computes and stores the average and current value '''
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val, n = 1) :
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [12]:
def train(train_loader, model, criterion, optimizer, epoch, logging = True):
    batch_time = AverageMeter()
    losses = AverageMeter()
    avg_score = AverageMeter()

    model.train()
    num_steps = len(train_loader)

    end = time.time()
    lr_str = ''

    for i, (input_, target) in enumerate(train_loader):
        if i >= num_steps:
            break
            
        confs_k = []
        predicts_k = [] 
        loss_k = []
        for k in range(4):
            output = model(input_.to(device), k)
            confs_c, predicts_c = torch.max(output.detach(), dim=1)
            confs_k.append(confs_c)
            predicts_k.append(predicts_c)
            
            target_k = (target>k).long()
            loss_c = criterion(output, target_k.to(device))
            loss_k.append(loss_c)
            losses.update(loss_c.data.item(), input_.size(0))
        
        loss = sum(loss_k)
        confs = sum(confs_k)/len(confs_k)
        predicts = sum(predicts_k)
        
        
        avg_score.update(GAP(predicts.detach(), confs, target))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if logging and i % LOG_FREQ == 0:
            print(f'{epoch} [{i}/{num_steps}]\t'
                        f'time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                        f'loss {losses.val:.4f} ({losses.avg:.4f})\t'
                        f'GAP {avg_score.val:.4f} ({avg_score.avg:.4f})'
                        + lr_str)
        if has_time_run_out():
            break

    print(f' * average GAP on train {avg_score.avg:.4f}')
    return avg_score.avg

In [13]:
def inference(data_loader, model):
    ''' Returns predictions and targets, if any. '''
    model.eval()

    activation = nn.Softmax(dim=1)
    all_predicts, all_confs, all_targets = [], [], []

    with torch.no_grad():
        for i, data in enumerate(tqdm(data_loader)):
            if data_loader.dataset.mode != 'test':
                input_, target = data
            else:
                input_, target = data, None

            predicts_k = []
            confs_k = []
            for k in range(4):
                output = model(input_.to(device), k)
                output = activation(output)
                confs_c, predicts_c = torch.topk(output, 1)
                predicts_k.append(predicts_c)
                confs_k.append(confs_c)
            
            predicts = sum(predicts_k)
            confs = sum(confs_k)/len(confs_k)
            all_confs.append(confs)
            all_predicts.append(predicts)

            if target is not None:
                all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None

    return predicts, confs, targets

In [14]:
def test(test_loader, model):
    predicts, targets = inference(test_loader, model)
    predicts = predicts.cpu().numpy().flatten()
    targets = targets.cpu().numpy().flatten()
    return cohen_kappa_score(targets, predicts)

In [15]:
def train_loop(epochs, train_loader, test_loader, model, criterion, optimizer,
               validate=True):
    train_res = []

    test_res = []
    for epoch in trange(1, epochs + 1):
        start_time = time.time()
        train_acc = train(train_loader, model, criterion, optimizer, epoch, logging=True)
        if has_time_run_out():
            break
        train_res.append(train_acc)

        if validate:
            test_acc = test(test_loader, model)
            test_res.append(test_acc)
            print(f"validation score: {test_acc}")
            
    return train_res, test_res

In [16]:
def has_time_run_out() -> bool:
    return time.time() - global_start_time > TIME_LIMIT - 1000

# training

In [17]:
train_dataset = ImageDataset(train_df, mode='train')
val_dataset = ImageDataset(val_df, mode='val')
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          drop_last=True, num_workers=NUM_WORKERS)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=NUM_WORKERS)

In [18]:
model = ORCNN()
model = nn.DataParallel(model)
model = model.to(device)

Loaded pretrained weights for efficientnet-b4


In [23]:
model.load_state_dict(torch.load("../model/orcnn.ptm"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
global_start_time = time.time()
train_res, test_res = train_loop(NUM_EPOCHS, train_loader, val_loader, model, criterion, optimizer)

In [24]:
predicts, confs, targets = inference(val_loader, model)

100%|██████████| 446/446 [04:27<00:00,  2.46it/s]


In [28]:
print(classification_report(targets.cpu(), predicts.cpu()))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89      1969
           1       0.20      0.12      0.15       205
           2       0.64      0.47      0.54       460
           3       0.56      0.61      0.58       130
           4       0.66      0.57      0.61        65

   micro avg       0.78      0.78      0.78      2829
   macro avg       0.58      0.54      0.56      2829
weighted avg       0.75      0.78      0.76      2829



In [25]:
confusion_matrix(targets.cpu(), predicts.cpu())

array([[2250,   46,   35,    0,    0],
       [ 130,   78,   71,    0,    0],
       [  95,   54,  431,   76,    4],
       [   5,    1,   57,   95,   10],
       [   3,    3,   27,   19,   72]])

In [27]:
cohen_kappa_score(targets.cpu(), predicts.cpu(), weights="quadratic")

0.8593974269806929

In [40]:
torch.save(model.state_dict(), "../model/efficientnet_wc.ptm")